# Урок 4. Парсинг HTML. XPath

1)Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.news

Для парсинга использовать xpath. Структура данных должна содержать:

название источника,
наименование новости,
ссылку на новость,
дата публикации

In [151]:
from pymongo import MongoClient
from pprint import pprint
from lxml import html
import requests
from datetime import datetime
import re

In [152]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
          AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

In [153]:
client = MongoClient('localhost',27017)
db = client['news_db']
db_news = db.news

In [154]:
def request_mail():
    mail_news = []
    keys = ('source', 'title', '_id', 'date')
    date_format = '%Y-%m-%dT%H:%M:%S%z'

    mail_url = 'https://news.mail.ru'
    response = requests.get(mail_url, headers=header)
    dom = html.fromstring(response.text)

    names = dom.xpath("//a[@class='list__text']/text()| \
                      //span[@class='photo__title photo__title_new photo__title_new_hidden js-topnews__notification']/text()")

    for i in range(len(names)):
        names[i] = names[i].replace(u'\xa0', u' ')


    links = dom.xpath("//a[contains(@class, 'js-topnews__item')]/@href|//a[@class='list__text']/@href")
    i = 0


    for i in range(len(links)):
        if links[i].find('https') == -1:
            links[i] = mail_url + links[i]

    dates = []
    sources = []

    for link in links:
        request_news = requests.get(link, headers=header)
        root = html.fromstring(request_news.text)
        date = root.xpath("//span[@class='note__text breadcrumbs__text js-ago']/@datetime")
        date = datetime.strptime(date[0], date_format)
        source = root.xpath("//a[@class='link color_gray breadcrumbs__link']//span[@class='link__text']/text()")
        dates.append(date)
        sources.extend(source)

    for item in list(zip(sources, names, links, dates)):
        news_dict = {}
        for key, value in zip(keys, item):
            news_dict[key] = value
        db_news.replace_one({'_id' : news_dict['_id']}, news_dict, True)

In [164]:
def request_lenta():
    date_format = ' %H:%M, %d %b %Y'

    lenta_url = 'https://lenta.ru'
    response = requests.get(lenta_url, headers=header)
    dom = html.fromstring(response.text)

    def month_from_ru_to_eng(date):
        date = date.replace('января', 'jan')
        date = date.replace('декабря', 'dec')
        date = date.replace('февраля', 'feb')
        date = date.replace('марта', 'mar')
        date = date.replace('апреля', 'apr')
        date = date.replace('мая', 'may')
        date = date.replace('июня', 'jun')
        date = date.replace('июля', 'jul')
        date = date.replace('августа', 'aug')
        date = date.replace('сентября', 'sep')
        date = date.replace('октября', 'oct')
        date = date.replace('ноября', 'nov')
        date = date.replace('декабря', 'dec')
        return date

    items = dom.xpath("//section[contains(@class, 'js-top-seven')]//div[contains(@class, 'item')]")
    
    for item in items:
        news = {}
        name = item.xpath(".//a/text()")
        link = item.xpath(".//a/@href")
        date = item.xpath(".//a/time/@datetime")

        for i in range(len(link)):
            if link[i].find('https') == -1:
                news['_id'] = lenta_url + link[i]
            else:
                news['_id'] = link[i]
        news['source'] = 'lenta.ru'
        news['title'] = name[0].replace(u'\xa0', u' ')
        news['date'] = datetime.strptime(month_from_ru_to_eng(date[0]), date_format)
        db_news.replace_one({'_id' : news['_id']}, news, True)

In [170]:
def request_yandex():
    yandex_news = []
    date_format = '%H:%M'

    yandex_url = 'https://yandex.ru/news'
    response = requests.get(yandex_url, headers=header)
    dom = html.fromstring(response.text)

    items = dom.xpath("//div[@class='stories-set stories-set_main_no stories-set_pos_1']//td")
    for item in items:
        news = {}
        source_date = item.xpath(".//div[@class='story__date']/text()")[0].replace(u'\xa0', u' ')

        mm = source_date[-2:]
        hh = source_date[-5:-3]

        now = datetime.now()
        if  'вчера' not in source_date:
            date = datetime(now.year, now.month, now.day, int(hh), int(mm), 0)
            source = source_date[:-6]
        else:
            news['date'] = datetime(now.year, now.month, now.day.timedelta(days=1), int(hh), int(mm), 0)
            source = source_date[:-14]

        name = item.xpath(".//a/text()")
        link = item.xpath(".//a/@href")

        news['source'] = source
        news['title'] = name[0].replace(u'\xa0', u' ')
        for i in range(len(link)):
            if link[i].find('https') == -1:
                news['_id'] = yandex_url + link[i]
            else:
                news['_id'] = link[i]
        news['date'] = date
        
        db_news.replace_one({'_id' : news['_id']}, news, True)

2)Сложить все новости в БД

In [166]:
request_mail()
for i in db_news.find({}):
    pprint(i)

{'_id': 'https://news.mail.ru/politics/42452596/',
 'date': datetime.datetime(2020, 7, 5, 11, 23, 53),
 'source': 'РИА Новости',
 'title': 'Путин рассказал о недостатках в Конституции СССР'}
{'_id': 'https://news.mail.ru/incident/42454934/',
 'date': datetime.datetime(2020, 7, 5, 16, 39, 35),
 'source': 'Lenta.Ru',
 'title': 'В Китае объявили третий уровень опасности из-за бубонной чумы'}
{'_id': 'https://sportmail.ru/news/biathlon/42453328/',
 'date': datetime.datetime(2020, 7, 5, 12, 23, 38),
 'source': 'ТАСС',
 'title': 'ДНК мужчины, обнаруженная в пробе Зайцевой, принадлежит ее мужу'}
{'_id': 'https://news.mail.ru/politics/42450563/',
 'date': datetime.datetime(2020, 7, 5, 10, 32, 58),
 'source': 'Коммерсантъ',
 'title': 'Экс-премьер ДНР: республики Донбасса скоро станут частью России'}
{'_id': 'https://news.mail.ru/society/42452968/',
 'date': datetime.datetime(2020, 7, 5, 11, 30, 52),
 'source': 'Новости Mail.ru',
 'title': 'Актер Вилле Хаапасало стал продавать хачапури (фото, ви

In [167]:
request_lenta()
for i in db_news.find({}):
    pprint(i)

{'_id': 'https://news.mail.ru/politics/42452596/',
 'date': datetime.datetime(2020, 7, 5, 11, 23, 53),
 'source': 'РИА Новости',
 'title': 'Путин рассказал о недостатках в Конституции СССР'}
{'_id': 'https://news.mail.ru/incident/42454934/',
 'date': datetime.datetime(2020, 7, 5, 16, 39, 35),
 'source': 'Lenta.Ru',
 'title': 'В Китае объявили третий уровень опасности из-за бубонной чумы'}
{'_id': 'https://sportmail.ru/news/biathlon/42453328/',
 'date': datetime.datetime(2020, 7, 5, 12, 23, 38),
 'source': 'ТАСС',
 'title': 'ДНК мужчины, обнаруженная в пробе Зайцевой, принадлежит ее мужу'}
{'_id': 'https://news.mail.ru/politics/42450563/',
 'date': datetime.datetime(2020, 7, 5, 10, 32, 58),
 'source': 'Коммерсантъ',
 'title': 'Экс-премьер ДНР: республики Донбасса скоро станут частью России'}
{'_id': 'https://news.mail.ru/society/42452968/',
 'date': datetime.datetime(2020, 7, 5, 11, 30, 52),
 'source': 'Новости Mail.ru',
 'title': 'Актер Вилле Хаапасало стал продавать хачапури (фото, ви

In [171]:
request_yandex()
for i in db_news.find({}):
    pprint(i)

{'_id': 'https://news.mail.ru/politics/42452596/',
 'date': datetime.datetime(2020, 7, 5, 11, 23, 53),
 'source': 'РИА Новости',
 'title': 'Путин рассказал о недостатках в Конституции СССР'}
{'_id': 'https://news.mail.ru/incident/42454934/',
 'date': datetime.datetime(2020, 7, 5, 16, 39, 35),
 'source': 'Lenta.Ru',
 'title': 'В Китае объявили третий уровень опасности из-за бубонной чумы'}
{'_id': 'https://sportmail.ru/news/biathlon/42453328/',
 'date': datetime.datetime(2020, 7, 5, 12, 23, 38),
 'source': 'ТАСС',
 'title': 'ДНК мужчины, обнаруженная в пробе Зайцевой, принадлежит ее мужу'}
{'_id': 'https://news.mail.ru/politics/42450563/',
 'date': datetime.datetime(2020, 7, 5, 10, 32, 58),
 'source': 'Коммерсантъ',
 'title': 'Экс-премьер ДНР: республики Донбасса скоро станут частью России'}
{'_id': 'https://news.mail.ru/society/42452968/',
 'date': datetime.datetime(2020, 7, 5, 11, 30, 52),
 'source': 'Новости Mail.ru',
 'title': 'Актер Вилле Хаапасало стал продавать хачапури (фото, ви